In [ ]:
# ostatnie piwo: https://www.polskikraft.pl/piwo/9740
# ostatni browar: https://www.polskikraft.pl/browar/411
# ostatni user: https://www.polskikraft.pl/profil/28348

In [2]:
import os
dumpdir = '/content/drive/MyDrive/datasets/PolskiKraft'

In [3]:
import pandas as pd
from tqdm.auto import tqdm

# scraping

In [272]:
!mkdir browary
!mkdir piwa

## breweries

In [ ]:
for i in tqdm(range(1, 413)):
    ! wget --quiet https://www.polskikraft.pl/browar/{i} -O browary/{i}.html

In [285]:
!zip -q -r browary.zip browary
!cp browary.zip {dumpdir}
!ls -lh {dumpdir}

total 2.6M
-rw------- 1 root root 2.6M Apr 10 02:18 browary.zip


## beers

In [23]:
# for i in tqdm(range(1, 9740)):
#     ! wget --quiet https://www.polskikraft.pl/piwo/{i} -O piwa/{i}.html

# this would tahe ages (8-9h), lets go somewhat parallel

In [ ]:
import subprocess
MAX = 9740

i = 511
pbar = tqdm(total=MAX)
pbar.update(i)
while True:
    N = 50
    ls_output = [subprocess.Popen([
        "wget", "--quiet",
        f"https://www.polskikraft.pl/piwo/{j}",
        "-O", f"piwa/{j}.html"
    ]) for j in range(i, i+N)]
    pbar.set_postfix({'tasks': 'queued'})
    [x.communicate() for x in ls_output] # async wait
    pbar.set_postfix({'tasks': 'received'})
    i += N
    pbar.update(N)
    if i > MAX:
        break
pbar.close()
# now it will take about 1.5h, seems fine.
# This should finish by 6:30, 10th of april.
# Lets hope no one uses their app so early on Saturdays.

In [24]:
!ls -lth piwa | wc -l

510


In [ ]:
!zip -q -r piwa.zip piwa
!cp piwa.zip {dumpdir}
!ls -lh {dumpdir}

In [ ]:
# for i in tqdm(range(1, 9740+1)):
#     if not os.path.exists(f'piwa/{i}.html'):
#         !wget --quiet https://www.polskikraft.pl/piwo/{i} -O piwa/{i}.html

# for i in tqdm(range(9740+1 9900)):
#     if os.path.exists(f'piwa/{i}.html'):
#         !rm piwa/{i}.html

# Extracting the data

In [ ]:
# import requests
# URL = 'https://www.monster.com/jobs/search/?q=Software-Developer&where=Australia'
# page = requests.get(URL)

from bs4 import BeautifulSoup

with open('browary/411.html', 'r') as f:
    html_content = f.read()

In [121]:
soup = BeautifulSoup(html_content, 'html.parser')

# Brewery

In [ ]:
def extract_brewery_beers_from_brewery_html(html_content):
    #in: string containing html file
    #out: list of dicts of extracted properties:
    #     [{...}]
    soup = BeautifulSoup(html_content, 'html.parser')
    beer_entries = soup.find_all("div", {"class": "pk-search-entry-link"})
    results = []
    for html_entry in beer_entries:

        tmp = html_entry.find_all("div", {"class":"row pk-search-entry"})[0].contents
        results.append({
            'brewery_name': soup.find_all("ol", {"class":"breadcrumb pk-breadcrumb"})[0].find_all('li')[1].text,
            'name': tmp[3].text,
            'rating': float(html_entry.find_all("div", {"class":"row rating"})[0].text),
            'type': tmp[6].text,
            'thumbnail_link': tmp[1].get('src'),
        })
    return results

In [ ]:
pd.DataFrame(extract_brewery_beers_from_html(html_content))

,brewery_name,name,rating,type,thumbnail_link
0,Monkey Browar,Wróg U Bram,3.6,American Pale Ale,https://www.polskikraft.pl/img/photos/16019238...
1,Monkey Browar,Born To Be Hoppy,4.1,DDH DIPA,https://www.polskikraft.pl/img/photos/16019237...
2,Monkey Browar,Born To Be Hoppy,4.4,DDH Double IPA,https://www.polskikraft.pl/img/photos/16019229...


In [ ]:
def extract_brewery_rating_from_brewery_html(html_content):
    #in: string containing html file
    #out: dict of extracted properties:
    #     {...}
    soup = BeautifulSoup(html_content, 'html.parser')
    the_div = soup.find_all("div", {"class":"panel-body pk-panel-photo-tile"})[0]
    results = {
        'name': the_div.find("p", {"class":"beer-name"}).text,
        'beers': the_div.find("p", {"class":"brewery-name"}).text,
        'rating': float(the_div.find("span", {"class":"rating"}).text),
        'logo_link': soup.find_all("div", {"class":"panel-body pk-panel-photo-tile"})[0].find('img').get('src'),
    }
    return results

In [ ]:
extract_brewery_rating_from_brewery_html(html_content)

{'beers': '3 piwa',
 'logo_link': 'https://www.polskikraft.pl/img/photos/1601922522.jpg',
 'name': 'Monkey Browar',
 'rating': 4.0}

# Beers

In [ ]:
with open('piwa/9740.html', 'r') as f:
    html_content = f.read()
soup = BeautifulSoup(html_content, 'html.parser')

In [270]:
def extract_similar_beers_from_beer_html(html_content):
    #in: string containing html file
    #out: dict of extracted properties:
    #     {...}

    soup = BeautifulSoup(html_content, 'html.parser')
    similar_div = soup.find_all("div", {"class":"panel-body pk-panel-tile pk-panel-tile-list"})[4]
    hrefs = [x.get("href") for x in similar_div.find_all("a", {"class":"big-link"})]
    return [{'similar': x.split('_')[-1], 'similar_link': x} for x in hrefs]

In [271]:
pd.DataFrame(extract_similar_beers_from_beer_html(html_content))

,similar,similar_link
0,1748,https://www.polskikraft.pl/piwo/pinta-vermont-...
1,2330,https://www.polskikraft.pl/piwo/browar-ziemia-...
2,2476,https://www.polskikraft.pl/piwo/browar-bazylis...
3,2485,https://www.polskikraft.pl/piwo/browar-deer-be...
4,2568,https://www.polskikraft.pl/piwo/piwne-podziemi...
...,...,...
294,9700,https://www.polskikraft.pl/piwo/piwojad-test-%...
295,9704,https://www.polskikraft.pl/piwo/browar-stu-mos...
296,9705,https://www.polskikraft.pl/piwo/piwojad-ddh-ne...
297,9723,https://www.polskikraft.pl/piwo/alebrowar-new-...


In [267]:
def extract_beer_data_from_beer_html(html_content):
    #in: string containing html file
    #out: dict of extracted properties:
    #     {...}

    soup = BeautifulSoup(html_content, 'html.parser')
    the_div = soup.find_all("div", {"class":"panel-body pk-panel-photo-tile"})[0]
    description_div = soup.find_all("div", {"class":"panel-body pk-panel-tile"})[0]
    results = {
        'name': the_div.find("p", {"class":"beer-name"}).text,
        'brewery': the_div.find("p", {"class":"brewery-name"}).text,
        'rating': float(the_div.find("span", {"class":"rating"}).text),
        'logo_link': soup.find_all("div", {"class":"panel-body pk-panel-photo-tile"})[0].find('img').get('src'),
        'features': {x.find("p").text: x.find("h1").text
            for x in description_div.find_all("div", {"class": "col-xs-4"})},
        'descriptive_features': {x.find("h3").text: x.find("p").text.strip()
            for x in description_div.find_all("div", {"class": "col-xs-12 pk-details-category"})}
    }
    return results

In [268]:
extract_beer_data_from_beer_html(html_content)

{'brewery': 'BROWAR ABSZTYFIKANT',
 'descriptive_features': {'CHMIELE': 'Citra (USA), Mosaic (USA), Simcoe (USA)',
  'STYL': 'New England IPA'},
 'features': {'alc': '5.6', 'blg': '14', 'ibu': '-'},
 'logo_link': 'https://www.polskikraft.pl/img/photos/1607626847.jpg',
 'name': 'Jupikajej 6.0',
 'rating': 3.1}

In [214]:
def extract_user_ratings_from_beer_html(html_content):
    #in: string containing html file
    #out: list of dicts of extracted properties:
    #     {...}
    soup = BeautifulSoup(html_content, 'html.parser')
    the_div = soup.find("div", {"class":"panel panel-default hidden-xs"})
    ratings_element = the_div.find_all("div", {"class":"row pk-comments-item pk-comment-link"})
    results = []
    for user_rating_element in ratings_element:
        user_profile_link = user_rating_element.find_all("a")[1].get('href')
        username_element = user_rating_element.find("p", {"class": "username"})
        tmp = user_rating_element.find_all("div", {"class":"row pk-comments-bjcp-bar"})
        results.append({
            'rating': len(user_rating_element.find_all("i", {"class":"fa fa-star fa-lg"})),
            'user': list(username_element.children)[0].strip(),
            'date': username_element.find("sub").text,
            'user_id': user_profile_link.split('/')[-1],
            'user_profile': user_profile_link,
            'user_avatar_link': user_rating_element.find("img").get("src"),
            'comment': user_rating_element.find("p", {"class":"content"}).text.strip(),
            'BJCP': {
                x.find('p', {"class":"title"}).text:
                x.find('p', {"class":"rating"}).text
                for x in tmp
                },
        })
    return results

In [215]:
pd.DataFrame(extract_user_ratings_from_beer_html(html_content))

,rating,user,date,user_id,user_profile,user_avatar_link,comment,BJCP
0,3,eMeS,12/02/2021,2201,https://www.polskikraft.pl/profil/2201,https://graph.facebook.com/v2.5/13347658665491...,,{}
1,3,Tomasz Romaszko,13/12/2020,21290,https://www.polskikraft.pl/profil/21290,https://graph.facebook.com/v2.5/22950427405543...,,{}
2,3,WaldoBeerSan,12/12/2020,14866,https://www.polskikraft.pl/profil/14866,https://www.polskikraft.pl/img/layout/ic_user_...,"Dobre Piwo, słodycz niska, barwa bursztynowa, ...","{'AROMAT': '8/12', 'WYGLĄD': '3/3', 'SMAK': '1..."
3,3,Piter Casch,10/12/2020,13459,https://www.polskikraft.pl/profil/13459,https://platform-lookaside.fbsbx.com/platform/...,Wygląd wody z kałuży. W smaku ok.,{}


# Users

In [ ]:
# In theory the data in user htmls is redundant to what we already have
# there should be no need to scrap the user profiles.

# It would be a huge burden on the PolskiBrowar.pl servers to 
# handle 20k+ of our requests downloading user profiles
# in order not to disrupt their service (it would take at least 3h, 
# in the worst case 23h) I do not even implement 
# the user-data extracting code.
# At least for now, beware.